# PRÁCTICA 1 VISUALIZACIÍON DE DATOS

Tenemos varios campos:

COUNTRY:país 

AMOUNT: un volumen de ventas en litros del producto.

SUBBRAND: el producto de la venta

YEAR: Año 

MONTH: Mes

SCENARIO: (dos valores)
- AI forecast: venta predicha en algún momento
- Actual: que se ha producido de verdad

Forecast: Si scenario es actual, este valor es Null. Indica el mes en periodos. Ejemplos:
- AI_P02F es Enero (a mes vencido). La prediccion se hizo a finales de enero
- AI_PF: Es como si fuera el mes de diciembre (AI_P13F) pero este dato no existe.

## Bibliotecas y Funciones

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
meses_dict = {
    'AI_P01F': 1,
    'AI_P02F': 2,
    'AI_P03F': 3,
    'AI_P04F': 4,
    'AI_P05F': 5,
    'AI_P06F': 6,
    'AI_P07F': 7,
    'AI_P08F': 8,
    'AI_P09F': 9,
    'AI_P10F': 10,
    'AI_P11F': 11,
    'AI_P12F': 12,
    'AI_PF': 12
}

## Carga de datos

In [3]:
# Cargar el archivo CSV
ruta_archivo = 'datos_ejercicio_ventas.csv'  # Reemplaza 'nombre_del_archivo.csv' por el nombre de tu archivo o la ruta completa
df = pd.read_csv(ruta_archivo)

# Mostrar las primeras filas del dataframe
print(df.head())


         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT  
0         2023.0  754356.237194  
1         2023.0  560030.558029  
2         2023.0   88501.980847  
3         2023.0  363224.511516  
4         2023.0  396176.120491  


## 1. Exploración

Responder a:
1. nº de actuals y forecast
2. horizonte de previsión de los forecast: Nº de puntos que predices
3. nº de paises y de productos:
4. Que histórico de actuals o de forecast
5. Forecast distintos

In [4]:
print(df.dtypes)

COUNTRY           object
SUBBRAND          object
YEAR               int64
MONTH              int64
SCENARIO          object
FORECAST          object
FORECAST_YEAR    float64
AMOUNT           float64
dtype: object


### 1.1 Nº de actuals, Nº de forecast

In [5]:
conteo_scenarios = df['SCENARIO'].value_counts()
# Crear etiquetas personalizadas para mostrar el nombre y el número total
labels = [f'{scenario}: {count}' for scenario, count in zip(conteo_scenarios.index, conteo_scenarios.values)]

# Crear el gráfico de tarta con plotly.express
fig = px.pie(values=conteo_scenarios.values, 
             names=labels, 
             title="Distribución de Actuals y AI forecast",
             hole=0.3)  # 'hole=0.3' crea un gráfico tipo 'donut'

# Mostrar el gráfico
fig.show()

### 1.2 Horizonte de predicción

In [6]:
# Aplicar la transformación a la columna 'FORECAST', usando el diccionario
df1 = df
df1['FORECAST'] = df['FORECAST'].map(meses_dict)

# Convertir YEAR, MONTH, FORECAST, y FORECAST_YEAR a enteros (si no lo son ya)
df1['YEAR'] = pd.to_numeric(df1['YEAR'], errors='coerce').astype('Int64')
df1['MONTH'] = pd.to_numeric(df1['MONTH'], errors='coerce').astype('Int64')
df1['FORECAST'] = pd.to_numeric(df1['FORECAST'], errors='coerce').astype('Int64')
df1['FORECAST_YEAR'] = pd.to_numeric(df1['FORECAST_YEAR'], errors='coerce').astype('Int64')

# Crear nueva columna de fecha con YEAR y MONTH, asegurando que sean números válidos
df1['Fecha'] = pd.to_datetime(df1['YEAR'].astype(str) + '-' + df1['MONTH'].astype(str) + '-28', errors='coerce')

# Crear nueva columna de fecha con FORECAST_YEAR y FORECAST, asegurando que sean números válidos
df1['Fecha_forecast'] = pd.to_datetime(df1['FORECAST_YEAR'].astype(str) + '-' + df1['FORECAST'].astype(str) + '-28', errors='coerce')

# Eliminar las columnas YEAR, MONTH, FORECAST y FORECAST_YEAR
df1 = df1.drop(columns=['YEAR', 'MONTH', 'FORECAST', 'FORECAST_YEAR'])

# Mostrar las primeras filas del dataframe para verificar
print(df1.head())


         COUNTRY        SUBBRAND     SCENARIO         AMOUNT      Fecha  \
0       Portugal     Lipton (L3)  AI_forecast  754356.237194 2023-12-28   
1  Great Britain     Lipton (L3)  AI_forecast  560030.558029 2023-12-28   
2          Spain  Pepsi Max (L3)  AI_forecast   88501.980847 2023-12-28   
3  Great Britain        7up (L3)  AI_forecast  363224.511516 2024-12-28   
4        Hungary     Lipton (L3)  AI_forecast  396176.120491 2023-09-28   

  Fecha_forecast  
0     2023-02-28  
1     2023-10-28  
2     2023-09-28  
3     2023-10-28  
4     2023-03-28  


In [7]:
# Generamos la copia del dataframe
df1_h = df1.copy()

# Calcular horizonte de predicción en meses solo para filas donde el SCENARIO es 'AI_forecast'
df1_h['horizonte_prediccion_meses'] = df1_h.apply(
    lambda row: ((row['Fecha'] - row['Fecha_forecast']).days / 30.44) if row['SCENARIO'] == 'AI_forecast' else None,
    axis=1
)

# Ordenar por el horizonte de predicción en meses y seleccionar los 20 valores más altos
top_20_horizonte = df1_h.nlargest(20, 'horizonte_prediccion_meses')

# Encontrar el valor máximo del horizonte de predicción en meses
max_horizonte = df1_h.loc[df1_h['horizonte_prediccion_meses'].idxmax()]
print(max_horizonte[['COUNTRY', 'SUBBRAND', 'SCENARIO', 'Fecha', 'Fecha_forecast', 'horizonte_prediccion_meses']])

COUNTRY                                    Norway
SUBBRAND                       Pepsi Regular (L3)
SCENARIO                              AI_forecast
Fecha                         2025-02-28 00:00:00
Fecha_forecast                2023-09-28 00:00:00
horizonte_prediccion_meses              17.049934
Name: 5, dtype: object


Por lo que el horizonte son 18 meses 

### 1.3 nº de países y de productos

In [8]:
# Número de países distintos y valores únicos
num_paises = df1['COUNTRY'].nunique()
valores_paises = ", ".join(df1['COUNTRY'].unique())
print(f"Número de países: {num_paises}")
print("Posibles valores de COUNTRY:")
print(f"- {valores_paises}\n")

# Número de productos distintos y valores únicos
num_productos = df1['SUBBRAND'].nunique()
valores_productos = ", ".join(df1['SUBBRAND'].unique())
print(f"Número de productos: {num_productos}")
print("Posibles valores de SUBBRAND:")
print(f"- {valores_productos}")




Número de países: 9
Posibles valores de COUNTRY:
- Portugal, Great Britain, Spain, Hungary, Norway, Denmark, Netherlands, Italy, Czech

Número de productos: 6
Posibles valores de SUBBRAND:
- Lipton (L3), Pepsi Max (L3), 7up (L3), Pepsi Regular (L3), Mountain Dew (L3), 7up Free (L3)


### 1.4 nº Forecast distintos

In [9]:
# Filtrar solo los registros de predicciones (AI_forecast)
df_forecast = df1[df1['SCENARIO'] == 'AI_forecast']

# Contar el número de fechas de forecast distintas
num_forecasts = df_forecast['Fecha_forecast'].nunique()

print("Número de predicciones (forecasts) distintas realizadas:", num_forecasts)

Número de predicciones (forecasts) distintas realizadas: 11


## 2. Distribución de las ventas realizadas

### 2.1 Por país

In [10]:
# Filtrar el DataFrame para obtener solo las ventas reales (Actual) y agrupar por país
df1_real = df1[df1['SCENARIO'].str.lower() == 'actual'].copy()
df1_real = df1_real.drop(['SCENARIO', 'Fecha_forecast'], axis=1)

# Corregir el nombre de la República Checa
df1_real['COUNTRY'] = df1_real['COUNTRY'].replace({'Czech': 'Czech Republic'})

# Agrupar por país y sumar las ventas
df_sales_by_country = df1_real.groupby('COUNTRY', as_index=False)['AMOUNT'].sum()

# Crear gráfico de burbujas
fig = px.scatter_geo(df_sales_by_country,
                     locations="COUNTRY",
                     locationmode="country names",
                     size="AMOUNT",
                     hover_name="COUNTRY",
                     hover_data={"AMOUNT": True},
                     projection="natural earth",
                     title="Distribución de Ventas totales realizadas en Europa")

# Configurar vista centrada en Europa
fig.update_geos(
    scope="europe",
    showcoastlines=True,
    coastlinecolor="Black",
    showcountries=True,
    countrycolor="Black"
)

# Mostrar gráfico
fig.show()


Ahora, vamos a ver cómo se distribuyen los productos las ventas una vez que ya tenemos una idea de la magnitud de consumo dependiendo del país

In [11]:
# Definir un diccionario de colores consistente para cada producto
color_dict = {
    'Lipton (L3)': '#FFD700',          # Amarillo para Lipton (té helado)
    'Pepsi Max (L3)': '#1E90FF',       # Azul claro para Pepsi Max
    '7up (L3)': '#006400',             # Verde oscuro para 7up
    'Pepsi Regular (L3)': '#00008B',   # Azul oscuro para Pepsi Regular
    'Mountain Dew (L3)': '#ADFF2F',    # Verde lima para Mountain Dew
    '7up Free (L3)': '#32CD32'         # Verde claro para 7up Free
}

# Crear gráfico de tarta para cada país con las marcas ordenadas de mayor a menor en sentido horario
for country in df1_real['COUNTRY'].unique():
    df_country = df1_real[df1_real['COUNTRY'] == country]
    
    # Agrupar las ventas totales por SUBBRAND y sumar AMOUNT
    df_grouped = df_country.groupby('SUBBRAND', as_index=False)['AMOUNT'].sum()
    
    # Ordenar las marcas de mayor a menor según AMOUNT
    df_grouped = df_grouped.sort_values('AMOUNT', ascending=False)
    
    # Crear gráfico de tarta con los colores consistentes y orden de las marcas
    fig = px.pie(df_grouped, 
                 names='SUBBRAND', 
                 values='AMOUNT', 
                 title=f'Distribución de Ventas por Producto en {country}',
                 color='SUBBRAND',
                 color_discrete_map=color_dict)
    
    # Actualizar el trazo para establecer la dirección y rotación
    fig.update_traces(sort=False,              # No ordenar automáticamente, usar el orden de los datos
                      direction='clockwise',   # Sentido horario
                      rotation=0)             # Iniciar en la parte superior (12 en punto)
    
    fig.show()


### 2.2. Cada mes y año

In [12]:
# Crear columnas separadas para el mes y el año
df1_real['Año'] = df1_real['Fecha'].dt.year
df1_real['Mes'] = df1_real['Fecha'].dt.month

# Agrupar los datos por mes y año, sumando las ventas (AMOUNT)
df_monthly_sales = df1_real.groupby(['Año', 'Mes'], as_index=False)['AMOUNT'].sum()

# Definir una paleta de colores pastel para los años 2023 y 2024
color_dict = {
    2023: '#AEC6CF',  # Azul pastel para 2023
    2024: '#FFB347'   # Naranja pastel para 2024
}

fig = go.Figure()

# Agregar datos de cada año con su color específico
for year in df_monthly_sales['Año'].unique():
    df_year = df_monthly_sales[df_monthly_sales['Año'] == year]
    fig.add_trace(
        go.Bar(
            x=df_year['Mes'],
            y=df_year['AMOUNT'],
            name=str(year),
            marker_color=color_dict[year]  # Aplicar el color pastel correspondiente
        )
    )

# Ajustar los nombres de los meses en el eje x y el diseño de la leyenda
fig.update_layout(
    title='Distribución de Ventas por Mes y Año',
    xaxis=dict(
        title='Mes',
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    ),
    yaxis_title='Ventas (Litros)',
    barmode='stack',
    legend_title_text='Año'
)

fig.show()

### 2.3 Cada marca

In [13]:
# Agrupar los datos por marca y sumar las ventas
df_sales_by_brand = df1_real.groupby('SUBBRAND', as_index=False)['AMOUNT'].sum()

# Definir el diccionario de colores
color_dict = {
    'Lipton (L3)': '#FFD700',          # Amarillo para Lipton (té helado)
    'Pepsi Max (L3)': '#1E90FF',       # Azul claro para Pepsi Max
    '7up (L3)': '#006400',             # Verde oscuro para 7up
    'Pepsi Regular (L3)': '#00008B',   # Azul oscuro para Pepsi Regular
    'Mountain Dew (L3)': '#ADFF2F',    # Verde lima para Mountain Dew
    '7up Free (L3)': '#32CD32'         # Verde claro para 7up Free
}

# Crear gráfico de barras con colores específicos para cada marca
fig = px.bar(df_sales_by_brand, 
             x='SUBBRAND', 
             y='AMOUNT', 
             title='Distribución de Ventas por Marca',
             labels={'AMOUNT': 'Ventas (Litros)', 'SUBBRAND': 'Marca'},
             color='SUBBRAND',
             color_discrete_map=color_dict)

fig.show()


## 3. tendencia y estacionalidad

### 3.1 País con menos ventas (todas las ventas)

Gracias al primer gráfico de burbujas se puede ver que el país con menos ventas es España, por lo que analizaremos la tendencia y la estacionalidad de este país en concreto.

In [14]:
# Filtrar los datos para obtener solo los registros de España
df1_RealEspana = df1_real[df1_real['COUNTRY'] == 'Spain'].copy()

# Eliminar la columna 'COUNTRY'
df1_RealEspana.drop(columns=['COUNTRY'], inplace=True)


# Agrupar las ventas totales por año y mes
df_monthly_sales_espana = df1_RealEspana.groupby(['Año', 'Mes'], as_index=False)['AMOUNT'].sum()

# Renombrar las columnas a los nombres requeridos por pd.to_datetime
df_monthly_sales_espana = df_monthly_sales_espana.rename(columns={'Año': 'year', 'Mes': 'month'})
df_monthly_sales_espana['day'] = 1  # Asignar el día 1 para cada registro

# Crear una columna de fecha continua
df_monthly_sales_espana['Fecha'] = pd.to_datetime(df_monthly_sales_espana[['year', 'month', 'day']])

# Crear gráfico de líneas para mostrar la tendencia mensual en una sola línea continua
fig = px.line(df_monthly_sales_espana, 
              x='Fecha', 
              y='AMOUNT', 
              title='Tendencia Mensual de Ventas en España',
              labels={'AMOUNT': 'Ventas Totales (Litros)', 'Fecha': 'Mes'})

# Crear una lista de fechas de inicio de cada mes como valores de ticks y convertir a array para evitar la advertencia
tickvals = np.array(pd.date_range(start=df_monthly_sales_espana['Fecha'].min(), 
                                  end=df_monthly_sales_espana['Fecha'].max(), 
                                  freq='MS'))  # 'MS' para inicio de mes

# Ajustar el eje x para que muestre el nombre completo de cada mes
fig.update_layout(
    xaxis=dict(
        tickformat="%B",  # Formato de nombre completo del mes (Enero, Febrero, etc.)
        tickmode="array",
        tickvals=tickvals  # Usar el array de fechas como valores de ticks
    ),
    yaxis_title='Ventas Totales (Litros)'
)

fig.show()






c:\Users\jbarc\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Ahora vemos la estacionalidad de españa

In [15]:
# Agrupar las ventas totales por año y mes
df_monthly_sales_espana = df1_RealEspana.groupby(['Año', 'Mes'], as_index=False)['AMOUNT'].sum()

# Crear gráfico de líneas para mostrar la tendencia mensual, con una línea por cada año
fig = px.line(df_monthly_sales_espana, 
              x='Mes', 
              y='AMOUNT', 
              color='Año', 
              title='Estacionalidad Mensual de Ventas en España',
              labels={'AMOUNT': 'Ventas Totales (Litros)', 'Mes': 'Mes'})

# Ajustar los nombres de los meses en el eje x y el diseño del gráfico
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    ),
    yaxis_title='Ventas (Litros)'
)

fig.show()


Por tanto se puede observar en españa en estos dos años una tendencia estable o muy poco creciente y una gran estacionalidad dependiendo de las vacaciones mayormente (en el 2023 la semana santa cayó en marzo y en 2024 en abril)

### 3.2 De la marca con más ventas (PepsiMax)

In [16]:
# Filtrar los datos para obtener solo los registros de Pepsi Max
df1_PepsiMax = df1_real[df1_real['SUBBRAND'] == 'Pepsi Max (L3)'].copy()

# Agrupar las ventas totales por año y mes
df_monthly_sales_pepsimax = df1_PepsiMax.groupby(['Año', 'Mes'], as_index=False)['AMOUNT'].sum()

# Renombrar las columnas a los nombres requeridos por pd.to_datetime
df_monthly_sales_pepsimax = df_monthly_sales_pepsimax.rename(columns={'Año': 'year', 'Mes': 'month'})
df_monthly_sales_pepsimax['day'] = 1  # Asignar el día 1 para cada registro

# Crear una columna de fecha continua
df_monthly_sales_pepsimax['Fecha'] = pd.to_datetime(df_monthly_sales_pepsimax[['year', 'month', 'day']])

# Crear gráfico de líneas para mostrar la tendencia mensual de ventas de Pepsi Max
fig = px.line(df_monthly_sales_pepsimax, 
              x='Fecha', 
              y='AMOUNT', 
              title='Tendencia Mensual de Ventas de Pepsi Max',
              labels={'AMOUNT': 'Ventas Totales (Litros)', 'Fecha': 'Mes'})

# Crear una lista de fechas de inicio de cada mes como valores de ticks
tickvals = np.array(pd.date_range(start=df_monthly_sales_pepsimax['Fecha'].min(), 
                                  end=df_monthly_sales_pepsimax['Fecha'].max(), 
                                  freq='MS'))  # 'MS' para inicio de mes

# Ajustar el eje x para que muestre el nombre completo de cada mes
fig.update_layout(
    xaxis=dict(
        tickformat="%B %Y",  # Formato de nombre completo del mes y año (Enero 2023, Febrero 2023, etc.)
        tickmode="array",
        tickvals=tickvals  # Usar el array de fechas como valores de ticks
    ),
    yaxis_title='Ventas Totales (Litros)'
)

fig.show()


c:\Users\jbarc\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [17]:
# Filtrar los datos para obtener solo los registros de Pepsi Max
df1_PepsiMax = df1_real[df1_real['SUBBRAND'] == 'Pepsi Max (L3)'].copy()

# Agrupar las ventas totales por año y mes
df_monthly_sales_pepsimax = df1_PepsiMax.groupby(['Año', 'Mes'], as_index=False)['AMOUNT'].sum()

# Crear gráfico de líneas para mostrar la estacionalidad mensual, con una línea por cada año
fig = px.line(df_monthly_sales_pepsimax, 
              x='Mes', 
              y='AMOUNT', 
              color='Año', 
              title='Estacionalidad Mensual de Ventas de Pepsi Max',
              labels={'AMOUNT': 'Ventas Totales (Litros)', 'Mes': 'Mes'})

# Ajustar los nombres de los meses en el eje x y el diseño del gráfico
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    ),
    yaxis_title='Ventas (Litros)'
)

fig.show()


En este caso si que se ve cierta tendencia positiva, y claramente una estacionalidad por país.


Como trabajo a futuro podría estudiarse de una forma más rigurosa la estacionalidad, pero para ello es necesario tener al menos 2 años completos, ya que la estacionalidad se verá en ese caso. Una opción podría ser usar los datos predichos en agosto del 2024 para los meses siguientes hasta diciembre del 2024 y usar esos datos para analizar este dato, aunque no sea estrictamente riguroso.

## 4. Predicciones en España y su calidad.

In [18]:
# Cambiar el nombre de la columna 'FORECAST' a 'FORECAST_MONTH'
df.rename(columns={'FORECAST': 'FORECAST_MONTH'}, inplace=True)

# Eliminar las columnas 'Fecha' y 'Fecha_forecast'
df.drop(columns=['Fecha', 'Fecha_forecast'], inplace=True)

print(df.head())

         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO  FORECAST_MONTH  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast               2   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast              10   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast               9   
3  Great Britain        7up (L3)  2024     12  AI_forecast              10   
4        Hungary     Lipton (L3)  2023      9  AI_forecast               3   

   FORECAST_YEAR         AMOUNT  
0           2023  754356.237194  
1           2023  560030.558029  
2           2023   88501.980847  
3           2023  363224.511516  
4           2023  396176.120491  


Primero hacemos una pequeña exploración de datos a ver si encontramos problemas (datos repetidos, registros vacíos etc)

In [19]:
# Exploración de los datos para identificar problemas

# 1. Comprobar si existen valores nulos
nulos = df.isnull().sum()
print("Cantidad de valores nulos por columna:\n", nulos)

# 2. Comprobar si existen registros duplicados
duplicados = df.duplicated().sum()
print("Cantidad de registros duplicados:", duplicados)

# 3. Resumen estadístico de las variables numéricas
resumen_estadistico = df.describe()
print("Resumen estadístico de las variables numéricas:\n", resumen_estadistico)

# 4. Comprobar valores únicos en la columna 'SCENARIO'
valores_scenario = df['SCENARIO'].unique()
print("Valores únicos en la columna 'SCENARIO':", valores_scenario)

# 5. Comprobar si hay registros con 'SCENARIO' como 'Actual' pero 'FORECAST_MONTH' no es nulo
problemas_forecast = df[(df['SCENARIO'] == 'Actual') & (df['FORECAST_MONTH'].notna())]
if not problemas_forecast.empty:
    print("Problemas con 'FORECAST_MONTH' para registros con 'SCENARIO' como 'Actual':\n", problemas_forecast)
else:
    print("No se encontraron problemas con 'FORECAST_MONTH' para registros con 'SCENARIO' como 'Actual'")

Cantidad de valores nulos por columna:
 COUNTRY             0
SUBBRAND            0
YEAR                0
MONTH               0
SCENARIO            0
FORECAST_MONTH    900
FORECAST_YEAR     900
AMOUNT              0
dtype: int64
Cantidad de registros duplicados: 678
Resumen estadístico de las variables numéricas:
               YEAR     MONTH  FORECAST_MONTH  FORECAST_YEAR        AMOUNT
count      18666.0   18666.0         17766.0        17766.0  1.866600e+04
mean   2023.716383  6.475463        7.593718         2023.0  9.721822e+05
std       0.590782  3.463632         3.26516            0.0  1.915283e+06
min         2023.0       1.0             2.0         2023.0 -2.171201e+05
25%         2023.0       3.0             5.0         2023.0  8.754541e+04
50%         2024.0       6.0             8.0         2023.0  3.081759e+05
75%         2024.0       9.0            11.0         2023.0  1.078576e+06
max         2025.0      12.0            12.0         2023.0  1.481563e+07
Valores únicos en 

Bueno, vemos que hay 678 registros duplicados, para evitar este problema a la hora de ajustar el código vamos a eliminar uno de cada registro duplicado.

In [20]:
# Guardar una copia del DataFrame y eliminar duplicados
df2 = df.copy()
df2 = df2.drop_duplicates()

Procedemos ahora a quedarnos solo con los valores de españa.

In [21]:
# Filtrar los valores donde 'COUNTRY' es 'Spain' y eliminar la columna 'COUNTRY'
df2 = df2[df2['COUNTRY'] == 'Spain']
df2 = df2.drop(columns=['COUNTRY'])
print("DataFrame después de filtrar por 'Spain' y eliminar la columna 'COUNTRY':\n", df2.head())

DataFrame después de filtrar por 'Spain' y eliminar la columna 'COUNTRY':
               SUBBRAND  YEAR  MONTH     SCENARIO  FORECAST_MONTH  \
2       Pepsi Max (L3)  2023     12  AI_forecast               9   
20  Pepsi Regular (L3)  2023     12  AI_forecast               5   
25         Lipton (L3)  2025      3  AI_forecast              11   
62       7up Free (L3)  2024      1  AI_forecast               4   
68            7up (L3)  2024      4  AI_forecast               2   

    FORECAST_YEAR         AMOUNT  
2            2023   88501.980847  
20           2023  134268.151080  
25           2023    9702.217953  
62           2023   70144.329753  
68           2023   38882.921227  


### Análisis de Calidad

Para descubrir de cuanta calidad son las predicciones podemos hacer un estudio del Error absoluto medio dependiendo de la longevidad desde la predicción hasta la fecha predicha.

In [23]:
# 1 Primero separamos los datos para poder manejarlos en dos dataframes
actuals = df2[df2['SCENARIO'] == 'actual'].copy()
forecasts = df2[df2['SCENARIO'] == 'AI_forecast'].copy()

# 2 Calcular el delta de predicción 
# Convertir columnas a numérico
cols_to_numeric = ['FORECAST_YEAR', 'FORECAST_MONTH', 'YEAR', 'MONTH']
forecasts[cols_to_numeric] = forecasts[cols_to_numeric].apply(pd.to_numeric)

# Calcular el delta de predicción (en meses)
forecasts['DELTA_PREDICCION'] = (
    (forecasts['YEAR'] - forecasts['FORECAST_YEAR']) * 12 +
    (forecasts['MONTH'] - forecasts['FORECAST_MONTH'])
)

# Filtrar deltas de predicción válidos (mayores que cero)
forecasts = forecasts[forecasts['DELTA_PREDICCION'] > 0]


In [24]:
# 3 Unir predicciones con datos reales en SUBBRAND, YEAR, MONTH
merged_df = forecasts.merge(
    actuals,
    on=['SUBBRAND', 'YEAR', 'MONTH'],
    how='inner',
    suffixes=('_forecast', '_actual')
)


In [25]:
# 4 Calcular el error absoluto porcentual
merged_df['ABS_PERCENTAGE_ERROR'] = ((
    (merged_df['AMOUNT_forecast'] - merged_df['AMOUNT_actual']) /
    merged_df['AMOUNT_actual']
) * 100).abs()

# 5 Agregar Errores por Delta de Predicción
# Agrupar por delta de predicción y calcular el error medio
error_by_delta = merged_df.groupby('DELTA_PREDICCION')['ABS_PERCENTAGE_ERROR'].mean().reset_index()

# **Filtrar los datos hasta el mes 15**
error_by_delta = error_by_delta[error_by_delta['DELTA_PREDICCION'] <= 15]



In [26]:
# 7 Graficar resultados
fig = px.line(
    error_by_delta,
    x='DELTA_PREDICCION',
    y='ABS_PERCENTAGE_ERROR',
    title='Evolución del error dependiendo de los meses desde la predicción',
    labels={'DELTA_PREDICCION': 'Meses desde la Predicción', 'ABS_PERCENTAGE_ERROR': 'Error Absoluto Porcentual Medio (%)'}
)
fig.show()

Esto nos confirma cuanto error podemos esperar en situaciones normales dependiendo de a cuanto tiempo lo estemos mirando.

Sin embargo, empezando en un error del 13.64% parece bastante alto, aunque si que es verdad que tratar de predecir estos datos puede ser complicado ya que tienen un comportamiento caótico. También, habría que comprobar el resultado usando otros algoritmos de predicción para ver si se puede llegar a un Error absoluto más bajo.

Hemos graficado solo hasta el mes 15 porque para deltas más altos contamos con pocas predicciones y por lo tanto no se puede generalizar tan correctamente el error absoluto medio.